In [1]:
just_checking_integrity=False

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [4]:
items_features = pd.read_csv('../dataset/processed_data/simplified_features_and_categories_30.csv')
items_features.head()

,item_id,feature_idx
0,19021,0
1,19021,22
2,19021,15
3,19021,446
4,19021,1


In [5]:
items_features['item_id'].max()

23691

In [6]:
dataset = np.zeros(
    (items_features['item_id'].max()+1, items_features['feature_idx'].max()+1)
    )
dataset[(items_features['item_id'], items_features['feature_idx'])] = 1

dataset
train_dataset=dataset[1:,:]

In [7]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(train_dataset,test_size=0.2,random_state=941116)

In [8]:
dataset.shape

(23692, 636)

In [9]:
train.shape

(18952, 636)

In [10]:
test.shape

(4739, 636)

# Autoencoder

In [11]:
latent_dimension = 64
starting_dimension = 512

inputs = layers.Input(int(items_features['feature_idx'].max()+1))

x = inputs 
dimension = starting_dimension

while(dimension != latent_dimension):
    #x = layers.BatchNormalization()(x)
    x = layers.Dense(units=dimension, activation='gelu',activity_regularizer=tf.keras.regularizers.L1L2(l1=1e-5,l2=1e-5))(x)
    x = layers.Dropout(0.1)(x)

    dimension = dimension//2

x = layers.Dense(units=dimension,activation="tanh",activity_regularizer=tf.keras.regularizers.L1L2(l1=5e-5,l2=4e-4))(x)
outputs = x
encoder = tf.keras.Model(inputs=inputs, outputs=outputs, name='encoder')

2024-02-13 18:00:00.428339: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
max_width = 1024
ending_dimension = items_features['feature_idx'].max()+1

inputs = layers.Input(latent_dimension)

x = inputs 
dimension = latent_dimension

x = layers.Dense(units=256, activation='gelu',activity_regularizer=tf.keras.regularizers.L1L2(l1=1e-5,l2=1e-5))(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(units=512, activation='gelu',activity_regularizer=tf.keras.regularizers.L1L2(l1=1e-5,l2=1e-5))(x)
x = layers.Dropout(0.2)(x)
    

x = layers.Dense(units=ending_dimension,activation="sigmoid")(x)
outputs = x

decoder = tf.keras.Model(inputs=inputs, outputs=outputs, name='decoder')

#decoder.summary()

In [13]:
ae = tf.keras.Model(inputs=encoder.inputs, outputs=decoder(encoder.outputs), name='auto_encoder')
ae.summary()

Model: "auto_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 636)]             0         
                                                                 
 dense (Dense)               (None, 512)               326144    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)              

In [14]:
def custom_loss(x_true,x_pred):
    return tf.keras.losses.cosine_similarity(x_true,x_pred)+tf.keras.losses.mean_squared_error(x_true,x_pred)#

In [15]:
ae.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.006), 
    loss=custom_loss,
    metrics=[tf.keras.metrics.CosineSimilarity(),tf.keras.metrics.MeanAbsoluteError()]
    )

# Training

In [16]:
ae.fit(
    train,
    train, 
    batch_size=64,
    validation_data=(test,test),
    epochs=5 if just_checking_integrity else 200,
    shuffle=True,
    callbacks=[
            tf.keras.callbacks.EarlyStopping(
                monitor='cosine_similarity',
                patience=10,
                restore_best_weights=True,
                mode="max"
            ),
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_cosine_similarity',
                factor=0.7,
                patience=2,
                min_lr=5e-5,
                mode="max",
                min_delta=0.0001,
            )
        ]
    )

Epoch 1/200
297/297 [==============================] - 12s 29ms/step - loss: -0.7581 - cosine_similarity: 0.7941 - mean_absolute_error: 0.0496 - val_loss: -0.8463 - val_cosine_similarity: 0.8700 - val_mean_absolute_error: 0.0315 - lr: 0.0060
Epoch 2/200
297/297 [==============================] - 8s 27ms/step - loss: -0.8551 - cosine_similarity: 0.8779 - mean_absolute_error: 0.0299 - val_loss: -0.8775 - val_cosine_similarity: 0.8979 - val_mean_absolute_error: 0.0247 - lr: 0.0060
Epoch 3/200
297/297 [==============================] - 8s 27ms/step - loss: -0.8767 - cosine_similarity: 0.8969 - mean_absolute_error: 0.0257 - val_loss: -0.8938 - val_cosine_similarity: 0.9116 - val_mean_absolute_error: 0.0234 - lr: 0.0060
Epoch 4/200
297/297 [==============================] - 8s 28ms/step - loss: -0.8888 - cosine_similarity: 0.9074 - mean_absolute_error: 0.0233 - val_loss: -0.9033 - val_cosine_similarity: 0.9201 - val_mean_absolute_error: 0.0204 - lr: 0.0060
Epoch 5/200
297/297 [==============

In [17]:
temp=encoder(dataset)

In [18]:
temp.shape

TensorShape([23692, 64])

In [19]:
temp

<tf.Tensor: shape=(23692, 64), dtype=float32, numpy=
array([[-0.02508243, -0.16864689,  0.00922986, ..., -0.10116268,
         0.00389111, -0.06173776],
       [-0.14411213,  0.12000129,  0.04996723, ...,  0.2083879 ,
        -0.02535575,  0.14935218],
       [-0.09013742, -0.04864637, -0.10119569, ...,  0.08597583,
        -0.0609637 ,  0.04487924],
       ...,
       [-0.11393449,  0.06673345, -0.05424234, ...,  0.10798514,
        -0.17017786, -0.08226722],
       [-0.2413577 , -0.01255779, -0.15978378, ..., -0.02866031,
        -0.20678058, -0.13705134],
       [ 0.00107431,  0.05251387, -0.0875941 , ..., -0.06650019,
        -0.05241397,  0.01870323]], dtype=float32)>

In [20]:
tf.reduce_sum(temp,axis=1).numpy().max(),tf.reduce_sum(temp,axis=1).numpy().min(),tf.reduce_sum(temp,axis=1).numpy().std()

(3.7147017, -3.985494, 0.910715)

In [21]:
np.save('../dataset/processed_data/compressed_features', encoder(dataset))